# Homework 5 : WeatherPy using Openweathermap API and Citipy

In [1]:
import openweathermapy.core as ow
from citipy import citipy as cp
import pandas as pd
import numpy as np
import random
import requests as req
import json
import matplotlib.pyplot as plt
import seaborn as sns
import time

%matplotlib inline

In [2]:
# Save config information.
#api_key = "bfe8624363d9fae0c73d21055be1bfe8" #my weather api
api_key = "25bc90a1196e6f153eece0bc0b0fc9eb" 
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"

# Build partial query URL
query_url = url + "appid=" + api_key + "&units=" + units + "&q="            

In [39]:
# Create dictionary of random coordinates; testing shows that range of 1500 will give <800 unique cities from Citipy
lat = []
lon = []
random_coords = {'Latitude': lat,'Longitude': lon}
for i in range(0,30): 
    ran_lat = random.uniform(-90, 90)
    lat.append(ran_lat)  
    ran_lon = random.uniform(-180,180)
    lon.append(ran_lon)

In [40]:
# Check number of coordinates
len(random_coords['Latitude'])

30

In [41]:
#Create dataframe for coordinates
df = pd.DataFrame(random_coords)
df.head(3)

,Latitude,Longitude
0,67.483246,132.932337
1,50.518862,-26.111528
2,68.359989,-21.021805


In [42]:
# Create additional colums for City, Temperature (F), Humidity (%), Cloudiness (%), Wind Speed (mph)
df['City'] = ""
df['Country'] = ""
df['Temperature (F)'] = ""
df['Humidity (%)'] = ""
df['Cloudiness (%)'] = ""
df['Wind Speed (mph)'] = ""
df.head(2)

,Latitude,Longitude,City,Country,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,67.483246,132.932337,,,,,,
1,50.518862,-26.111528,,,,,,


In [43]:
# Loop through coordinates in df with Citipy to return cities
row_count = 1
for index, row in df.iterrows():
   
    city = cp.nearest_city((row["Latitude"]), (row["Longitude"]))
    return_city = city.city_name
    return_country = city.country_code
    
    df.set_value(index, "City", return_city)
    df.set_value(index, "Country", return_country) 
     
    # Print log to ensure loop is working correctly
    #print("Now retrieving city # " + str(row_count) + " " + return_city + ", " + return_country)
    row_count += 1 
    
#df.head(3)

In [44]:
# Check number of unique (non-repeating) cities were generated
len(df["City"].unique())

29

In [45]:
# Visual check and length check duplicated cities
#df.sort_values("City", inplace=True)
mask=df["City"].duplicated()
df[mask].head()
#len(df[mask])

,Latitude,Longitude,City,Country,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
27,72.830314,108.114637,khatanga,ru,,,,


In [46]:
# There are alot of duplicates since Citipy tries to match to random coords to nearest city
# Drop duplicates and check that results count is same as unique and reset index numbers
df.drop_duplicates(subset = ["City"], keep = 'first', inplace=True) # 'first keeps first occurence 
df.reset_index(drop=True, inplace=True)
df

,Latitude,Longitude,City,Country,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,67.483246,132.932337,verkhoyansk,ru,,,,
1,50.518862,-26.111528,lagoa,pt,,,,
2,68.359989,-21.021805,illoqqortoormiut,gl,,,,
3,21.458411,76.701493,nepa nagar,in,,,,
4,69.602653,-84.317581,clyde river,ca,,,,
5,31.761028,117.921291,chaohu,cn,,,,
6,31.610945,-109.167375,douglas,us,,,,
7,9.969122,92.681076,port blair,in,,,,
8,-36.975060,65.105995,souillac,mu,,,,
9,-32.480452,143.147510,broken hill,au,,,,


In [47]:
# Create function to retrieve, set, and print data for each city
def retrieve_set_data():
    print("Now retrieving Set: %s, Record: %s, City: %s: %s" %
          (str(set_count), str(index), city.title(), target_url))
            
    weather_data = [req.get(target_url).json()]
    return_temp = weather_data[0]["main"]["temp_max"]
    return_humid = weather_data[0]["main"]["humidity"]
    return_cloud = weather_data[0]["clouds"]["all"]
    return_wind = weather_data[0]["wind"]["speed"]
    return_lat = weather_data[0]["coord"]["lat"]
    return_lon = weather_data[0]["coord"]["lon"]
    
    df.set_value(index, "Temperature (F)", return_temp)
    df.set_value(index, "Humidity (%)", return_humid) 
    df.set_value(index, "Cloudiness (%)", return_cloud)
    df.set_value(index, "Wind Speed (mph)", return_wind)         
    df.set_value(index, "Latitude", return_lat)
    df.set_value(index, "Longitude", return_lon)

In [48]:
# Loop through all cities using OpenWeatherMap to return weather info
# Need to overwrite random-generated coords with actual coords since plotting random-generated lat not same as city lat
row_count = 1
set_count = 1
for index, row in df.iterrows():
    
    city = row["City"]
    target_url = (query_url + city)
    
    # Print log to ensure loop is working correctly
    # Retreive in increments include timer to mitigate exceeding API limits
    if row_count<=5:
        retrieve_set_data()
        row_count += 1 
        
    else:
        retrieve_set_data()
        row_count = 1
        set_count += 1
        time.sleep(5)

Now retrieving Set: 1, Record: 0, City: Verkhoyansk: http://api.openweathermap.org/data/2.5/weather?appid=25bc90a1196e6f153eece0bc0b0fc9eb&units=imperial&q=verkhoyansk
Now retrieving Set: 1, Record: 1, City: Lagoa: http://api.openweathermap.org/data/2.5/weather?appid=25bc90a1196e6f153eece0bc0b0fc9eb&units=imperial&q=lagoa
Now retrieving Set: 1, Record: 2, City: Illoqqortoormiut: http://api.openweathermap.org/data/2.5/weather?appid=25bc90a1196e6f153eece0bc0b0fc9eb&units=imperial&q=illoqqortoormiut
Now retrieving Set: 1, Record: 3, City: Nepa Nagar: http://api.openweathermap.org/data/2.5/weather?appid=25bc90a1196e6f153eece0bc0b0fc9eb&units=imperial&q=nepa nagar
Now retrieving Set: 1, Record: 4, City: Clyde River: http://api.openweathermap.org/data/2.5/weather?appid=25bc90a1196e6f153eece0bc0b0fc9eb&units=imperial&q=clyde river
Now retrieving Set: 1, Record: 5, City: Chaohu: http://api.openweathermap.org/data/2.5/weather?appid=25bc90a1196e6f153eece0bc0b0fc9eb&units=imperial&q=chaohu
Now re

In [49]:
df

,Latitude,Longitude,City,Country,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,67.54,133.38,verkhoyansk,ru,57.25,95,92,5.17
1,39.05,-27.98,lagoa,pt,71.65,100,24,5.39
2,41.90,12.49,illoqqortoormiut,gl,64.4,63,0,2.24
3,21.47,76.40,nepa nagar,in,82,88,88,9.64
4,70.47,-68.59,clyde river,ca,37.4,93,90,3.36
5,31.60,117.87,chaohu,cn,78.8,100,20,8.95
6,54.15,-4.48,douglas,us,57.2,87,20,10.29
7,11.67,92.75,port blair,in,77.5,100,92,11.65
8,-20.52,57.52,souillac,mu,75.2,83,40,11.41
9,-31.95,141.43,broken hill,au,56.89,53,12,14.12
